## cryptoGRAPHy 1

> By sahuang
>
> Graphs have gained an increasing amount of attention in the world of Cryptography. They are used to model many real-world problems ranging from social media to traffic routing networks. Designing a secure Graph Encryption Scheme (GES) is important as querying plaintext graph database can leak sensitive information about the users.
> 
> In this challenge I have implemented a novel GES. Please help me verify if the cryptosystem works.
>
> Note: `lib.zip` remains unchanged in this series. The flag for this challenge will be used to access the next one when unlocked.
> 
> `nc chals.sekai.team 3001`
> 
> Attachment: lib.zip server.py

The source code given is quite dense, but what matters is that we are given some sort of AES key at the start and asked to obtain information from some ciphertext. Reading the source tells us we need to use the front 16 bytes of the key and then decyrpt chunks of the ciphertext 32 bytes at a time using the provided `utils.SymmetricDecrypt`. This reveals information about the IDs of the nodes visited in order.

In [10]:
from pwn import *
from libgraph import utils
from tqdm.auto import tqdm

r = remote("chals.sekai.team", 3001)

r.recvuntil(b"Key: ")
sk = bytes.fromhex(r.recvline().strip().decode())
sk_ske, sk_des = sk[:16], sk[16:]

def do_round():
    r.recvuntil(b"/50: ")

    # Pull starting node ID in plaintext
    src, _ = [int(i) for i in r.recvline().strip().decode().split()] 
    r.recvuntil(b"Response: ")
    response_full = bytes.fromhex(r.recvline().strip().decode())

    path = [src]
    for i in range(0, len(response_full), 32):
        # Decrypt next node ID in path
        step_tuple = utils.SymmetricDecrypt(sk_ske, response_full[i:i+32]) 
        step_tuple = [int(i) for i in step_tuple.decode().split(",")]
        path += [step_tuple[0]]
    r.sendline(" ".join(str(i) for i in path).encode())

for _ in tqdm(range(50)):
    do_round()
print(r.recvuntil(b"}"))

[x] Opening connection to chals.sekai.team on port 3001
[x] Opening connection to chals.sekai.team on port 3001: Trying 34.148.151.228
[+] Opening connection to chals.sekai.team on port 3001: Done


  0%|          | 0/50 [00:00<?, ?it/s]

b'> Original query: [+] Flag: SEKAI{GES_15_34sy_2_br34k_kn@w1ng_th3_k3y}'


## cryptoGRAPHy 2

> By sahuang
>
> I am wondering what can be leaked in my GES. Let me know if you can recover the graph structure in an Honest-But-Curious setting.
> 
> Note: `lib.zip` remains unchanged in this series. The flag for this challenge will be used to access the next one when unlocked.
> 
> `nc chals.sekai.team 3062`
>
> Attachment: server.py

We can query with a start node and end node and are given the "tokens" of each node on the way to destination. The challenge wants us to reproduce the "node degrees in the single-destination shortest path tree".

Query the paths of each starting node to the fixed destination node, generate a list of edges connecting adjacent tokens in all paths and re-build a "tree of tokens" (instead of tree of node IDs) with this edge list. Finally calculate the degree of each node and sort accordingly.

In [11]:
from pwn import *
from tqdm.auto import tqdm

r = remote("chals.sekai.team", 3062)
r.sendline(b"SEKAI{GES_15_34sy_2_br34k_kn@w1ng_th3_k3y}")

def do_round():
    # Recieve destination
    r.recvuntil(b"Destination: ")
    dst = int(r.recvline().strip().decode())

    # Query all paths from * -> destination
    for i in range(130):
        if i == dst:
            continue
        r.sendline(f"{i},{dst}".encode())

    # Recover adjacency lists
    token_adj = defaultdict(set)
    for i in range(130):
        if i == dst:
            continue
        r.recvuntil(b"Token: ")
        token = r.recvline().strip().decode()
        r.recvuntil(b"Query Response: ")
        trace = r.recvline().strip().decode()
        trace = trace[:64]

        token_adj[token].add(trace)
        token_adj[trace].add(token)
    
    # Calculate degrees
    proof = " ".join([str(i) for i in sorted([len(i) for i in token_adj.values()])])
    r.sendline(f"-1,-1\n{proof}".encode())

for _ in tqdm(range(10)):
    do_round()
print(r.recvuntil(b"}"))

[x] Opening connection to chals.sekai.team on port 3062
[x] Opening connection to chals.sekai.team on port 3062: Trying 34.148.151.228
[+] Opening connection to chals.sekai.team on port 3062: Done


  0%|          | 0/10 [00:00<?, ?it/s]

b'> Query u,v: [!] Invalid query!\n> Answer: [+] Flag: SEKAI{3ff1c13nt_GES_4_Shortest-Path-Queries-_-}'


## cryptoGRAPHy 3

> By sahuang
>
> Here is the hardest part: Can you directly recover the shortest path query if you are the server, having access to the original graph and all queries? (On a side note, this setting is somewhat realistic in scenarios such as _Google Maps_, where the whole routing map is available to the adversary.)
> 
> Note: `lib.zip` remains unchanged in this series. This is the last challenge.
> 
> `nc chals.sekai.team 3023`
>
> Attachment: server.py

We are given an edge list with original node IDs of a tree graph, all the lists of tokens for all possible shortest paths in a tree graph and need to find a way to map tokens back to their original node ID.

For each shortest path, the last 32-byte token uniquely identifies the "single source" of the shortest path. We can group all of the shortest paths using by the "single source token", then reconstruct the tree graph like in part 2 to obtain 60 different trees. All of these trees represent the same tree but with different a different root. We now need to find a way to convert trees with different root to the original node IDs.

First, find a node that can be uniquely identified from the tree structure, such as selecting the node with largest out-degree (and/or reconnecting until there is only 1 such node). Next, re-root each tree at that unqiue node. Ensure children are visited in the same order by implementing some kind of subtree hash function[^1]. Repeat this one last time using the edge list given at the start. All the trees will now have the same structure and we can map tokens to original node IDs by finding its path from root (e.g. using preorder).

[^1]: https://codeforces.com/blog/entry/101010

In [7]:
from pwn import *

r = remote("chals.sekai.team", 3023)
r.sendline(b"SEKAI{3ff1c13nt_GES_4_Shortest-Path-Queries-_-}")
r.send(b"1\n2\n")

# Original edge list
r.recvuntil(b"Edges: ")
edges = safeeval.expr(r.recvline().strip().decode())

# All token shortest paths
r.recvuntil(b"Query Responses: \n")
apsp_tokens = r.recvuntil(b"============ MENU ============").strip().decode()
apsp_tokens = apsp_tokens.split("\n")[:-1]

# Map of family to adjacency lists
# graphs[graph_family][a] = set(nodes connected to a)
graphs = defaultdict(lambda: defaultdict(set))

for src, dst in edges:
    graphs["original"][src].add(dst)
    graphs["original"][dst].add(src)

[x] Opening connection to chals.sekai.team on port 3023
[x] Opening connection to chals.sekai.team on port 3023: Trying 34.148.151.228
[+] Opening connection to chals.sekai.team on port 3023: Done


In [8]:
from tqdm.auto import tqdm

def get_nodes_of_max_degree(g):
    maximal_degree = max(len(v) for v in g.values())
    nodes_with_maximal_degree = [k for k, v in g.items() if len(v) == maximal_degree]
    return nodes_with_maximal_degree

# Ensure we have 1 unique node to root with
assert len(get_nodes_of_max_degree(graphs["original"])) == 1

def chunk(x, bs=64):
    return [ x[i:i+bs] for i in range(0, len(x), bs) ]

for tokens_resp in apsp_tokens:
    tokens, _resp = tokens_resp.split(" ") # We can throw away resp
    tokens = chunk(tokens)
    family = tokens[-1] # Each tree is uniquely identified by the last token

    # Recover edge list
    for src, dst in zip(tokens, tokens[1:]):
        graphs[family][src].add(dst)
        graphs[family][dst].add(src)

def root_graph(g):
    root_node = get_nodes_of_max_degree(g)[0]

    # children[node] = [children sorted by subtree hash]
    children = {}

    # DFS and return subtree hash, while building children
    def dfs1(node): 
        children[node] = []
        children_hashes = []
        for other in g[node]:
            if other in children:
                # Already visited, skip
                continue
            children_hashes.append((dfs1(other), other))

        if len(children_hashes) == 0:
            return "()" # leaf case

        children_hashes.sort() # Sort children by subtree hash
        own_hash, children[node] = list(zip(*children_hashes))
        # Ensure children's subtrees are unique, otherwise the tree cannot be uniquely rooted
        assert len(set(own_hash)) == len(own_hash) 

        # Calculate subtree hash, thankfully only 60 nodes in this tree
        # so we can abuse strings
        # https://codeforces.com/blog/entry/101010
        return  "(" + "".join(own_hash) + ")"
    dfs1(root_node)

    # Recover preorder
    preorder = 0
    node_to_preorder = {}
    def dfs2(node):
        nonlocal preorder
        node_to_preorder[node] = preorder
        preorder += 1
        for child in children[node]:
            dfs2(child)
    dfs2(root_node)

    return node_to_preorder

# family_nodeid_to_preorder[family][token][]
family_token_to_preorder = {}
for family in tqdm(graphs):
    family_token_to_preorder[family] = root_graph(graphs[family])
preorder_to_nodeid = {v: k for k, v in family_token_to_preorder["original"].items()}

  0%|          | 0/61 [00:00<?, ?it/s]

In [9]:
def do_round():
    r.recvuntil(b"Token: ")
    token = r.recvline().strip().decode()
    r.recvuntil(b"Query Response: ")
    trace = r.recvline().strip().decode()

    parts = chunk(token + trace[:len(trace)//2])
    family = parts[-1]

    steps = []
    for part in parts:
        steps.append(preorder_to_nodeid[family_token_to_preorder[family][part]])
    ans = " ".join(str(i) for i in steps)
    r.sendline(ans.encode())

r.sendline(b"3")
for _ in tqdm(range(10)):
    do_round()
print(r.recvuntil(b"}"))

  0%|          | 0/10 [00:00<?, ?it/s]

b'> Original query: [+] Flag: SEKAI{Full_QR_Attack_is_not_easy_https://eprint.iacr.org/2022/838.pdf}'
